# Assignment 1

Deadline: 19.03.2025, 12:00 CET

<Add your name, student-id and emal address>

In [2]:
# Import standard libraries
import os
import sys
import timeit # To compute runtimes
from typing import Optional

# Import third-party libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import local modules
project_root = os.path.dirname(os.path.dirname(os.getcwd()))   # Change this path if needed
src_path = os.path.join(project_root, 'qpmwp-course\\src')
sys.path.append(project_root)
sys.path.append(src_path)
from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.constraints import Constraints
from optimization.optimization import Optimization, Objective
from optimization.optimization_data import OptimizationData
from optimization.quadratic_program import QuadraticProgram, USABLE_SOLVERS

## 1. Solver Horse Race

### 1.a)
(3 points)

Generate a Multivariate-Normal random dataset of dimension TxN, T=1000, N=100, and compute a vector of expected returns, q, and a covariance matrix, P, using classes ExpectedReturn and Covariance respectively.

In [8]:

# Set the dimensions
T = 1000  # Number of time periods
N = 100   # Number of assets

# Generate a random mean vector from a normal distribution
mean = np.random.rand(N)
# Generate a random covariance matrix
A = np.random.rand(N, N)
cov = np.dot(A, A.transpose())  # To ensure the covariance matrix is positive semi-definite

# Generate the Multivariate-Normal random dataset
data = np.random.multivariate_normal(mean, cov, T)

# Convert the dataset to a DataFrame for easier manipulation
df = pd.DataFrame(data, columns=[f'Asset_{i+1}' for i in range(N)])


# Compute the vector of expected returns (mean returns) from df
scalefactor = 1
expected_return = ExpectedReturn(method='arithmetic', scalefactor=scalefactor)
expected_return.estimate(X=df, inplace=True)

# Compute the covariance matrix from df
covariance = Covariance(method='pearson')
covariance.estimate(X=df, inplace=True)


# Display the results
print("Vector of expected returns (q):")
print(expected_return.vector)
# 
print("\nCovariance matrix (P):")
print(covariance.matrix)

Vector of expected returns (q):
Asset_1      0.439781
Asset_2      0.003854
Asset_3      0.557548
Asset_4      0.783066
Asset_5      1.106094
               ...   
Asset_96     0.404006
Asset_97     0.965377
Asset_98     0.908774
Asset_99     0.893983
Asset_100    0.675554
Length: 100, dtype: float64

Covariance matrix (P):
             Asset_1    Asset_2    Asset_3    Asset_4    Asset_5    Asset_6  \
Asset_1    39.935473  29.044137  28.259793  28.210666  31.966498  25.268396   
Asset_2    29.044137  37.301116  27.133329  27.146006  29.262580  26.448117   
Asset_3    28.259793  27.133329  31.835300  25.985145  29.862545  24.311638   
Asset_4    28.210666  27.146006  25.985145  34.438228  31.243505  22.975007   
Asset_5    31.966498  29.262580  29.862545  31.243505  40.212618  25.703519   
...              ...        ...        ...        ...        ...        ...   
Asset_96   24.296039  22.055025  22.256000  22.204950  25.772871  19.753052   
Asset_97   26.388233  26.401770  23.257242

### 1.b)
(3 points)

Instantiate a constraints object by injecting column names of the data created in 1.a) as ids and add:
- a budget constaint (i.e., asset weights have to sum to one)
- lower bounds of 0.0 for all assets
- upper bounds of 0.2 for all assets
- group contraints such that the sum of the weights of the first 30 assets is <= 0.3, the sum of assets 31 to 60 is <= 0.4 and the sum of assets 61 to 100 is <= 0.5

In [9]:
# Instantiate the Constraints class
constraints = Constraints(ids = df.columns.tolist())

# Add budget constraint
constraints.add_budget(rhs=1, sense='=')

# Add box constraints (i.e., lower and upper bounds)
constraints.add_box(lower=0, upper=0.2)

# Add linear constraints
G = pd.DataFrame(np.zeros((3, N)), columns=constraints.ids)
G.iloc[0, :30] = 1
G.iloc[1, 30:60] = 1
G.iloc[2, 60:] = 1
h = pd.Series([0.3, 0.4, 0.5])
constraints.add_linear(G=G, sense='<=', rhs=h)


### 1.c) 
(4 points)

Solve a Mean-Variance optimization problem (using coefficients P and q in the objective function) which satisfies the above defined constraints.
Repeat the task for all open-source solvers in qpsolvers and compare the results in terms of:

- runtime
- accuracy: value of the primal problem.
- reliability: are all constarints fulfilled? Extract primal resisduals, dual residuals and duality gap.

Generate a DataFrame with the solvers as column names and the following row index: 'solution_found': bool, 'objective': float, 'primal_residual': float, 'dual_residual': float, 'duality_gap': float, 'runtime': float.

Put NA's for solvers that failed for some reason (e.g., unable to install the package or solvers throws an error during execution). 




In [10]:
list(USABLE_SOLVERS)

['quadprog', 'highs', 'cvxopt', 'qpalm', 'osqp', 'daqp']

In [11]:
import time
# Extract the constraints in the format required by the solver
GhAb = constraints.to_GhAb()

# Loop over solvers, instantiate the quadratic program, solve it and store the results
risk_aversion = 1

results = pd.DataFrame(columns=list(USABLE_SOLVERS), index={'solution_found': bool, 'objective': float, 'primal_residual': float, 'dual_residual': float, 'duality_gap': float, 'runtime': float})

for solver_name in USABLE_SOLVERS:
    qp = QuadraticProgram(
        P = covariance.matrix.to_numpy() * risk_aversion,
        q = expected_return.vector.to_numpy() * -1,
        G = GhAb['G'],
        h = GhAb['h'],
        A = GhAb['A'],
        b = GhAb['b'],
        lb = constraints.box['lower'].to_numpy(),
        ub = constraints.box['upper'].to_numpy(),
        solver = solver_name
    )

    start_time = time.time()
    qp.solve()
    end_time = time.time()

    sol = qp.results.get('solution')
    results.loc['solution_found', solver_name] = sol.found
    if sol.found:
        results.loc['objective', solver_name] = qp.objective_value()
        results.loc['primal_residual', solver_name] = sol.primal_residual()
        results.loc['dual_residual', solver_name] = sol.dual_residual()
        results.loc['duality_gap', solver_name] = sol.duality_gap()
        results.loc['runtime', solver_name] = end_time - start_time

Print and visualize the results

In [12]:
print(results)

                               quadprog                   highs  \
solution_found                     True                    True   
objective                      9.471288                9.471288   
primal_residual                     0.0                     0.0   
dual_residual                       0.0                     0.0   
duality_gap      2.4549529076267618e-11  1.5403572417582723e-08   
runtime                          0.0137                0.014194   

                                cvxopt                  qpalm  \
solution_found                    True                   True   
objective                     9.471289               9.470623   
primal_residual                    0.0               0.000027   
dual_residual                      0.0                    0.0   
duality_gap      6.056290111633444e-07  0.0006646425373500464   
runtime                       0.186897               0.012921   

                                osqp                    daqp  
solution_fo

## 2. Analytical Solution to Minimum-Variance Problem

(5 points)

- Create a `MinVariance` class that follows the structure of the `MeanVariance` class.
- Implement the `solve` method in `MinVariance` such that if `solver_name = 'analytical'`, the analytical solution is computed and stored within the object (if such a solution exists). If not, call the `solve` method from the parent class.
- Create a `Constraints` object by injecting the same ids as in part 1.b) and add a budget constraint.
- Instantiate a `MinVariance` object by setting `solver_name = 'analytical'` and passing instances of `Constraints` and `Covariance` as arguments.
- Create an `OptimizationData` object that contains an element `return_series`, which consists of the synthetic data generated in part 1.a).
- Solve the optimization problem using the created `MinVariance` object and compare the results to those obtained in part 1.c).


In [ ]:
# Define class MinVariance
from helper_functions import to_numpy

class MinVariance(Optimization):

    def __init__(self,
                 constraints: Constraints,
                 covariance: Optional[Covariance] = None,
                 **kwargs):
        super().__init__(
            constraints=constraints,
            **kwargs
        )
        self.covariance = Covariance() if covariance is None else covariance

    def set_objective(self, optimization_data: OptimizationData) -> None:
        X = optimization_data['return_series']
        covmat = self.covariance.estimate(X=X, inplace=False)
        self.objective = Objective(
            P = covmat
        )
        return None
    
    def solve(self) -> None:
        if self.params.get('solver_name') == 'analytical':
            constraints = self.constraints
            GhAb = constraints.to_GhAb()
            obj_coeff = self.objective.coefficients

            E = to_numpy(obj_coeff['P'])
            A = GhAb['A']
            b = GhAb['b']
            
            try:
                weights = np.linalg.inv(E) @ A.T @ np.linalg.inv(A @ np.linalg.inv(E) @ A.T) @ b
                self.results.update({
                    'weights': weights,
                    'status': True
                })        
                return None
            
            except ValueError as ve:
                print("ValueError:", ve)
                return super().solve()
            except Exception as e:
                print("Exception:", e)
                return super().solve()
        else:
            return super().solve()


# Create a constraints object with just a budget constraint
constraints = Constraints(ids = df.columns.tolist())
constraints.add_budget(rhs=[1], sense='=')

# Instantiate the MinVariance class
minvar = MinVariance(
    constraints=constraints,
    covariance=covariance,
    solver_name='analytical'
)

# Prepare the optimization data and prepare the optimization problem
optimization_data = OptimizationData(return_series=df)
minvar.set_objective(optimization_data=optimization_data)

# Solve the optimization problem and print the weights
minvar.solve()
print(minvar.results.get('weights'))

Shape of E: (100, 100)
Shape of A: (1, 100)
Shape of b: (1,)
[ 0.50658507 -1.89669468  2.54325499 -2.81260592  0.42090006 -2.36880135
 -1.70059944 -1.56137033 -0.45295373 -1.21109015 -0.18507463 -0.33415172
  0.34911333  0.2862648   0.27907706  0.59174764  1.11222381 -0.37063551
 -1.24564906 -1.16199774 -1.36576132 -1.01923137  1.08762055 -1.36689425
  0.68182603  1.8132053  -1.24939887  0.44880047 -0.31721993  0.45788032
 -2.16885082  0.08446289  0.03742658  5.07861746  0.54740781 -1.56658745
 -1.25378726 -0.96971097 -1.70255824 -2.02491766  0.86454987  0.26554575
 -0.49077351 -0.77776254  1.48515169  1.89438637  0.87526022  0.25394033
 -1.00896149  1.70555306 -0.71750965  2.2276285  -0.42254413  1.10379759
 -1.16146335 -0.25411908  3.76301596  0.77190189  0.77044523 -2.50004494
 -1.53914058 -1.01236718  1.99537578  0.78373767  0.20173625  0.92652714
  0.45775583  0.90882105 -1.23563221 -0.30711881 -0.79632278  0.85039097
  2.27625057  0.09620437 -0.22659631  2.30520779  1.6144982   1